In [50]:
from bs4 import BeautifulSoup
import re
import pandas as pd

In [51]:
#helper function to find lines in html document with DOI and titles of articles
def has_doi(href):
    return href and re.compile("chapter/").search(href)

In [52]:
def mining(html_doc, year, current_page, all_pages):
    #opening the html document (copy pasted and saved as a .doc file)
    doc = open(html_doc, "r", encoding = "utf-8") #ensuring I get correct characters! (issues with ')
    #parsing the document
    soup = BeautifulSoup(doc, 'html.parser' )

    list_of_doi = soup.find_all(href=has_doi)
        
    #getting the titles and the doi's from list generated helper function

    titles = []
    doi_str = []

    for element in list_of_doi:
        titles.append(element.get_text()) #returns the titles as the only text in the list
        string = str(element)
        first_substring = '/chapter'
        second_substring ='">'
        #separates out the DOIS
        doi_str.append(string[(string.find(first_substring)):string.find(second_substring)]) 
            
    ## now the lines containing author are found
    authors = soup.find_all("li", class_="c-author-list__item")
        
    #keeping only the author names
    authors_str = []
    for element in authors:
        string = str(element)
        first_substring = 'item">'
        second_substring ='</li>'
        authors_str.append(string[(string.find(first_substring)+6):string.find(second_substring)])
            
    #now the lines containing page numbers are found
    page_numbers= soup.find_all('div', class_ = "c-meta")
        
    #keeping only the page numbers
    page_numbers_str = []

    for element in page_numbers:
        string = element.get_text()[6:-1] #removes the white spaces and "Pages "
        both = string.split("-")
        if 'x' not in string: #filtering out front matters
            try: 
                if int(both[1])-int(both[0]) > 1: 
                    page_numbers_str.append(string)
            except:
                if "C" in string or "E" in string: #including corrections and erratum, are removed later
                    page_numbers_str.append(string)
    #filtering out back matters, only an issue in 2021 
    if year == 2021 and int(current_page) == int(all_pages):
        page_numbers_str = page_numbers_str[:-1]
                
    #a check that I've found the same amount of authors, titles, dois and page numbers
    #can be commented out if it works
    #print('page', len(page_numbers_str))
    #print('author', len(authors_str))
    #print(page_numbers_str)
    #if len(page_numbers_str) == len(authors_str) == len(titles) == len(doi_str):
    #    print("true")
    #else:
    #    print("You have an error!")
            
    #need to create a list of the year of publication to add to dataframe 
    year_of_pub = []
    for element in titles:
        year_of_pub.append(year)
    
    #creating the column names and content for the dataframe        
    data = {'Title': titles,
        'Authors': authors_str,
        'Page numbers' : page_numbers_str,
        'DOI': doi_str,
        'Year of publication' : year_of_pub
        }

    df = pd.DataFrame(data)

    return df 


In [53]:
#helper function to combine all df together
def data_together(data, year):
    combined_frame = pd.concat(data, ignore_index = True, sort = False)
    combined_frame.to_csv('database_miccai_'+ str(year) +'.csv')
    return combined_frame

In [54]:
#saving all 2012 together as one with the helper function above
miccai =['miccai 2012 part 1 page 1 of 5.doc', 'miccai 2012 part 1 page 2 of 5.doc', 
         'miccai 2012 part 1 page 3 of 5.doc' , 'miccai 2012 part 1 page 4 of 5.doc', 
         'miccai 2012 part 1 page 5 of 5.doc', 
         
         'miccai 2012 part 2 page 1 of 5.doc', 
         'miccai 2012 part 2 page 2 of 5.doc', 'miccai 2012 part 2 page 3 of 5.doc' ,
         'miccai 2012 part 2 page 4 of 5.doc', 'miccai 2012 part 2 page 5 of 5.doc', 
         
         
         'miccai 2012 part 3 page 1 of 5.doc', 'miccai 2012 part 3 page 2 of 5.doc', 
         'miccai 2012 part 3 page 3 of 5.doc' , 'miccai 2012 part 3 page 4 of 5.doc', 
         'miccai 2012 part 3 page 5 of 5.doc']

data = []
for element in miccai:  
    data.append(mining(element, 2012, element[24:25], element[29:30]))

data_together(data, 2012)

,Title,Authors,Page numbers,DOI,Year of publication
0,Reliable Assessment of Perfusivity and Diffusivity from Diffusion Imaging of the Body,"M. Freiman, S. D. Voss, R. V. Mulkern, J. M. Perez-Rossello, M. J. Callahan, Simon K. Warfield",1-9,/chapter/10.1007/978-3-642-33415-3_1,2012.0
1,Multi-organ Abdominal CT Segmentation Using Hierarchically Weighted Subject-Specific Atlases,"Robin Wolz, Chengwen Chu, Kazunari Misawa, Kensaku Mori, Daniel Rueckert",10-17,/chapter/10.1007/978-3-642-33415-3_2,2012.0
2,Radiation-Free Drill Guidance in Interlocking of Intramedullary Nails,"Benoit Diotte, Pascal Fallavollita, Lejing Wang, Simon Weidert, Peter-Helmut Thaller, Ekkehard E...",18-25,/chapter/10.1007/978-3-642-33415-3_3,2012.0
3,Developing Essential Rigid-Flexible Outer Sheath to Enable Novel Multi-piercing Surgery,"Siyang Zuo, Takeshi Ohdaira, Kenta Kuwana, Yoshihiro Nagao, Satoshi Ieiri, Makoto Hashizume et al.",26-33,/chapter/10.1007/978-3-642-33415-3_4,2012.0
4,Surgical Gesture Classification from Video Data,"Benjamín Béjar Haro, Luca Zappella, René Vidal",34-41,/chapter/10.1007/978-3-642-33415-3_5,2012.0
...,...,...,...,...,...
248,An Invariant Shape Representation Using the Anisotropic Helmholtz Equation,"A. A. Joshi, S. Ashrafulla, D. W. Shattuck, H. Damasio, R. M. Leahy",607-614,/chapter/10.1007/978-3-642-33454-2_75,2012.0
249,Phase Contrast Image Restoration via Dictionary Representation of Diffraction Patterns,"Hang Su, Zhaozheng Yin, Takeo Kanade, Seungil Huh",615-622,/chapter/10.1007/978-3-642-33454-2_76,2012.0
250,Context-Constrained Multiple Instance Learning for Histopathology Image Segmentation,"Yan Xu, Jianwen Zhang, Eric I-Chao Chang, Maode Lai, Zhuowen Tu",623-630,/chapter/10.1007/978-3-642-33454-2_77,2012.0
251,Structural-Flow Trajectories for Unravelling 3D Tubular Bundles,"Katerina Fragkiadaki, Weiyu Zhang, Jianbo Shi, Elena Bernardis",631-638,/chapter/10.1007/978-3-642-33454-2_78,2012.0


In [57]:
#saving all 2021 together as one with the helper function above
miccai =['miccai 2021 part 1 page 1 of 4.doc', 'miccai 2021 part 1 page 2 of 4.doc', 
         'miccai 2021 part 1 page 3 of 4.doc' , 'miccai 2021 part 1 page 4 of 4.doc', 
         
         'miccai 2021 part 2 page 1 of 4.doc', 'miccai 2021 part 2 page 2 of 4.doc', 
         'miccai 2021 part 2 page 3 of 4.doc' , 'miccai 2021 part 2 page 4 of 4.doc',
         
         'miccai 2021 part 3 page 1 of 4.doc', 'miccai 2021 part 3 page 2 of 4.doc', 
         'miccai 2021 part 3 page 3 of 4.doc' , 'miccai 2021 part 3 page 4 of 4.doc',
         
        'miccai 2021 part 4 page 1 of 4.doc', 'miccai 2021 part 4 page 2 of 4.doc', 
         'miccai 2021 part 4 page 3 of 4.doc' , 'miccai 2021 part 4 page 4 of 4.doc',
         
        'miccai 2021 part 5 page 1 of 5.doc', 'miccai 2021 part 5 page 2 of 5.doc', 
         'miccai 2021 part 5 page 3 of 5.doc' , 'miccai 2021 part 5 page 4 of 5.doc', 
         'miccai 2021 part 5 page 5 of 5.doc',
        
        'miccai 2021 part 6 page 1 of 4.doc', 'miccai 2021 part 6 page 2 of 4.doc', 
         'miccai 2021 part 6 page 3 of 4.doc' , 'miccai 2021 part 6 page 4 of 4.doc',
        
        'miccai 2021 part 7 page 1 of 5.doc', 'miccai 2021 part 7 page 2 of 5.doc', 
         'miccai 2021 part 7 page 3 of 5.doc' , 'miccai 2021 part 7 page 4 of 5.doc', 
         'miccai 2021 part 7 page 5 of 5.doc', 
        
        'miccai 2021 part 8 page 1 of 4.doc', 'miccai 2021 part 8 page 2 of 4.doc',
        'miccai 2021 part 8 page 3 of 4.doc', 'miccai 2021 part 8 page 4 of 4.doc']

data = []
for element in miccai:  
    data.append(mining(element, 2021, element[24:25], element[29:30]))
data_together(data, 2021)

,Title,Authors,Page numbers,DOI,Year of publication
0,Noisy Labels are Treasure: Mean-Teacher-Assisted Confident Learning for Hepatic Vessel Segmentation,"Zhe Xu, Donghuan Lu, Yixin Wang, Jie Luo, Jagadeesan Jayender, Kai Ma et al.",3-13,/chapter/10.1007/978-3-030-87193-2_1,2021
1,TransFuse: Fusing Transformers and CNNs for Medical Image Segmentation,"Yundong Zhang, Huiye Liu, Qiang Hu",14-24,/chapter/10.1007/978-3-030-87193-2_2,2021
2,Pancreas CT Segmentation by Predictive Phenotyping,"Yucheng Tang, Riqiang Gao, Hohin Lee, Qi Yang, Xin Yu, Yuyin Zhou et al.",25-35,/chapter/10.1007/978-3-030-87193-2_3,2021
3,Medical Transformer: Gated Axial-Attention for Medical Image Segmentation,"Jeya Maria Jose Valanarasu, Poojan Oza, Ilker Hacihaliloglu, Vishal M. Patel",36-46,/chapter/10.1007/978-3-030-87193-2_4,2021
4,Anatomy-Constrained Contrastive Learning for Synthetic Segmentation Without Ground-Truth,"Bo Zhou, Chi Liu, James S. Duncan",47-56,/chapter/10.1007/978-3-030-87193-2_5,2021
...,...,...,...,...,...
530,Weakly-Supervised Ultrasound Video Segmentation with Minimal Annotations,"Ruiheng Chang, Dong Wang, Haiyan Guo, Jia Ding, Liwei Wang",648-658,/chapter/10.1007/978-3-030-87237-3_62,2021
531,Content-Preserving Unpaired Translation from Simulated to Realistic Ultrasound Images,"Devavrat Tomar, Lin Zhang, Tiziano Portenier, Orcun Goksel",659-669,/chapter/10.1007/978-3-030-87237-3_63,2021
532,Visual-Assisted Probe Movement Guidance for Obstetric Ultrasound Scanning Using Landmark Retrieval,"Cheng Zhao, Richard Droste, Lior Drukker, Aris T. Papageorghiou, J. Alison Noble",670-679,/chapter/10.1007/978-3-030-87237-3_64,2021
533,Training Deep Networks for Prostate Cancer Diagnosis Using Coarse Histopathological Labels,"Golara Javadi, Samareh Samadi, Sharareh Bayat, Samira Sojoudi, Antonio Hurtado, Silvia Chang et al.",680-689,/chapter/10.1007/978-3-030-87237-3_65,2021
